In [1]:
import sys, os
import re
import nltk
from nltk.stem import PorterStemmer 
from collections import Counter
from nltk.corpus import stopwords
import math

In [2]:
ps = PorterStemmer() 
def words_count(file_name):
    with open (file_name,'r') as md_file:
        content = md_file.read()
    #remove punctuation
    content = re.sub('[^A-Za-z0-9]+', ' ', content)
    count = len(content)
    content = nltk.word_tokenize(content.lower())
    #remove stopwords
    content = [word for word in content if not word in stopwords.words()]
    for i in range(len(content)):
        #stemming words
        content[i] = ps.stem(content[i])
    file_dict = dict(Counter(content))
    for key, value in file_dict.items():
        #norm tfs
        file_dict[key] = value/count
    return file_dict

In [3]:
def words_count_f(folder_path):
    dirs = os.listdir(folder_path)
    file_dict = {}
    for file in dirs:
        #filter out trash file 
        if 'txt' in file:
            file_dict[file] = words_count(file)
    return file_dict

In [17]:
value_dict = words_count_f('/Users/mac/Documents/I427/pa1/')
#value_dict['slyfox.txt']

In [5]:
#define a new function return mapping_dict, contains index number and txt file names
def return_file(folder_path):
    dirs = os.listdir(folder_path)
    mapping_dict = dict(enumerate(dirs))
    return mapping_dict

mapping_dict = {}

#create an empty dict as argument 
inv_index = {}

def inv_indexer(mapping_dict, value_dict, inv_index):
    for key in value_dict.keys():
        if key not in mapping_dict.values():
            if not mapping_dict.keys():
                #the first entry, start with 0
                mapping_dict[1] = key
            else:
                #plus one at a time
                mapping_dict[int(max(mapping_dict.keys())+1)] = key
    for k, subdict in value_dict.items():
        for key, value in mapping_dict.items():
            #check which txt file it is
            if k == value:
                #loop through each term and its frequency
                for kk, v in subdict.items():
                    inv_index.setdefault(kk, {})[key] = v
    return inv_index, mapping_dict

In [6]:
mapping_dict = inv_indexer(mapping_dict, value_dict, inv_index)[1]
inv_index_dict = inv_indexer(mapping_dict, value_dict, inv_index)[0]

In [7]:
len(inv_index_dict)

1309

In [8]:
from itertools import islice
def take(n, iterable):
    "Return first n items of the iterable as a list"
    return list(islice(iterable, n))

In [9]:
sample = take(20, inv_index_dict.items())
sample

[('gim', {1: 0.00015216068167985393}),
 ('anoth',
  {1: 0.00015216068167985393,
   4: 0.00025031289111389235,
   6: 0.0005291005291005291}),
 ('cigarett', {1: 0.0006086427267194157}),
 ('mark', {1: 0.0030432136335970784}),
 ('extend', {1: 0.00015216068167985393}),
 ('hand', {1: 0.0004564820450395618, 2: 0.000142025280499929}),
 ('wait', {1: 0.00015216068167985393, 4: 0.00025031289111389235}),
 ('jami', {1: 0.002738892270237371}),
 ('drop',
  {1: 0.00030432136335970786,
   2: 0.000142025280499929,
   4: 0.00025031289111389235,
   5: 0.00013281976358082084,
   7: 0.00019704433497536947}),
 ('cancer', {1: 0.00030432136335970786}),
 ('dave', {1: 0.0025867315885575166}),
 ('said',
  {1: 0.0025867315885575166,
   2: 0.0007101264024996449,
   4: 0.000750938673341677,
   5: 0.0006640988179041042,
   7: 0.001379310344827586}),
 ('look',
  {1: 0.0009129640900791236,
   2: 0.000142025280499929,
   4: 0.000750938673341677,
   5: 0.0006640988179041042,
   7: 0.00019704433497536947}),
 ('mind', {1: 

## search common terms across files

In [10]:
lst = []
sub_dict = {}
ranking_dict = {}

def search_words(inv_index_dict):
    #get user input as a list
    user_input = list(input("Please enter the term(s) you want to search: ").strip().split())
    #check if all input terms exist in the dict
    if all(word in inv_index_dict.keys() for word in user_input):
        for word in user_input:
            #append the list of documents for each term to a large nested list
            lst.append([int(k) for k in inv_index_dict[word].keys()])
            #create a sub dict only contains terms we search
            sub_dict[word] = inv_index_dict[word]
        print(sub_dict)
        #find the intersection
        result = set.intersection(*[set(x) for x in lst]) 
        if result:
            for each in result:
                #use sub dict to sum the ranking values of each common key
                ranking_values = sum(d[each] for d in sub_dict.values())
                ranking_dict[mapping_dict[each]] = ranking_values
            #sort the dict by the ranking values in descending order
            for k in sorted(ranking_dict, key=ranking_dict.get, reverse=True):
                print(k, ranking_dict[k])
        else:
            print("Common documents don't exist.")
    else: 
        print("Invalid input! Please try again.")
    
search_words(inv_index_dict)

Please enter the term(s) you want to search: drop look said
{'drop': {1: 0.00030432136335970786, 2: 0.000142025280499929, 4: 0.00025031289111389235, 5: 0.00013281976358082084, 7: 0.00019704433497536947}, 'look': {1: 0.0009129640900791236, 2: 0.000142025280499929, 4: 0.000750938673341677, 5: 0.0006640988179041042, 7: 0.00019704433497536947}, 'said': {1: 0.0025867315885575166, 2: 0.0007101264024996449, 4: 0.000750938673341677, 5: 0.0006640988179041042, 7: 0.001379310344827586}}
blind.txt 0.003804017041996348
3pigs.txt 0.001773399014778325
3wishes.txt 0.0017521902377972466
slyfox.txt 0.001461017399389029
beautbst.txt 0.0009941769634995029


## all about tf-idf

In [11]:
tf_idf = {}

In [12]:
def tf_idf_q(folder_path):
    value_dict = words_count_f(folder_path)
    all_terms = []
    for each_file in value_dict.values():
        full_lst = list(each_file.keys())
        for each in full_lst:
            all_terms.append(each)
    #calculate idf for each unique term
    idf = dict(Counter(all_terms))
    for key, value in idf.items():
        idf[key] = 1/(1+math.log10(value))
    mapping_dict = {}
    for key in value_dict.keys():
        if key not in mapping_dict.values():
            if not mapping_dict.keys():
                mapping_dict[1] = key
            else:
                mapping_dict[int(max(mapping_dict.keys())+1)] = key
    for key in list(mapping_dict.values()):
        tf_idf[key] = sum([idf[each]*value_dict[key][each] for each in list(idf.keys())
                           if each in list(value_dict[key].keys())])
    user_input = list(input("Please enter your query: ").strip().split())
    #ranking
    for word in user_input:
        if 'rank' in word:
            ranking = sorted([(value,key) for (key,value) in tf_idf.items()],reverse=True)
            for each in ranking:
                print(each)
    #corresponding tf-idf score
    for file in value_dict.keys():
        for word in user_input:
            if word in file:
                sub_dict = {}
                sub_dict[file] = tf_idf[file]
                for each in sub_dict.items():
                    print(each)
    #check file with tf-idf score greater than certain number
    if '>' in user_input:
        for file, score in tf_idf.items():
            if score > float(user_input[user_input.index('>')+1]):
                zero_dict = {}
                zero_dict[file] = score
                for each in zero_dict.items():
                    print(each)

In [13]:
mapping_dict

{1: 'blind.txt',
 2: 'beautbst.txt',
 3: 'campfire.txt',
 4: '3wishes.txt',
 5: 'slyfox.txt',
 6: 'aircon.txt',
 7: '3pigs.txt'}

In [14]:
#e.g. 1
tf_idf_q('/Users/mac/Documents/I427/pa1/')

Please enter your query: show ranking
(0.08803210333334059, 'slyfox.txt')
(0.08006603659217393, 'aircon.txt')
(0.07942907218058502, 'blind.txt')
(0.07831145407433074, '3pigs.txt')
(0.07312373458844192, 'beautbst.txt')
(0.0730932215809103, 'campfire.txt')
(0.06929237548140457, '3wishes.txt')


In [15]:
#e.g. 2
tf_idf_q('/Users/mac/Documents/I427/pa1/')

Please enter your query: 3 fox
('3wishes.txt', 0.06929237548140457)
('slyfox.txt', 0.08803210333334059)
('3pigs.txt', 0.07831145407433074)


In [16]:
#e.g. 3
tf_idf_q('/Users/mac/Documents/I427/pa1/')

Please enter your query: > 0.077
('blind.txt', 0.07942907218058502)
('slyfox.txt', 0.08803210333334059)
('aircon.txt', 0.08006603659217393)
('3pigs.txt', 0.07831145407433074)
